In [23]:
import gensim
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
#from gensim import corpora, models
import pyLDAvis.gensim_models
import os
import pickle
from nltk.corpus import stopwords

In [13]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lukas\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [11]:
# with open('../data/restaurant_reviews.pkl', encoding='cp1252') as f:
#     df = pickle.load(f)


df = pd.read_csv('../data/restaurant_reviews.csv')
    
df.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date'],
      dtype='object')

In [20]:
# Load the regular expression library
import re
# Remove punctuation
df['text_processed'] = \
df['text'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
df['text_processed'] = \
df['text_processed'].map(lambda x: x.lower())
# Print out the first rows of papers
df['text_processed'].head()

<>:5: DeprecationWarning: invalid escape sequence \.
<>:5: DeprecationWarning: invalid escape sequence \.
C:\Users\lukas\AppData\Local\Temp/ipykernel_8316/2335645254.py:5: DeprecationWarning: invalid escape sequence \.
  df['text'].map(lambda x: re.sub('[,\.!?]', '', x))


0    this place has some of the best chinese take-o...
1    incredible donuts sometimes you have to go ear...
2    dont bother going to voodoo just come here ins...
3    the ramen here is less than great it came out ...
4    it's crazy how establishments on the west coas...
Name: text_processed, dtype: object

In [21]:
df.shape

(752143, 10)

In [22]:
docs=df['text_processed'].apply(word_tokenize)

KeyboardInterrupt: 

In [ ]:
d= gensim.corpora.Dictionary(docs)

#convert docs to vec
v = [d.doc2bow(doc) for doc in docs]

In [ ]:
os.environ.update({'MALLET_HOME':r'C:\Users\lukas\Documents\Projekte\yelp\mallet-2.0.8'})

mallet_path = r'C:\Users\lukas\Documents\Projekte\yelp\mallet-2.0.8\bin\mallet'

# ldamallet= gensim.models.LdaMallet(mallet_path, corpus = v, num_topics = 8, id2word = d)
ldamallet = gensim.models.ldamodel.LdaModel(corpus = v, num_topics = 8, id2word = d)
vecTop= ldamallet.show_topics()
for i in range(0, 8):
    print(vecTop[i])

In [ ]:
import joblib
#Save a model to disk, or reload a pre-trained model

joblib.dump(ldamallet, 'reviews_ldamallet.jl')

# # then reload it with
ldamallet_disk = joblib.load('reviews_ldamallet.jl')

In [ ]:
vecTop= ldamallet_disk.show_topics()
for i in range(0, 8):
    print(vecTop[i])

In [ ]:
#map the reviews data to the model, v is the reviews transformed to word vectors by doc2bow
m = ldamallet[v[0:len(v)]]

#assign topic to each review
import operator 
topic = []
for x in m:
    #find the topic with the highest proportions
    t = max(x, key = operator.itemgetter(1))
    topic.append(t[0])
    

In [ ]:
#map topic names to the topic numbers
topic_dict = {0:'Atmosphere', 1:'Food', 2:'Service',
             3:'Food', 4:'Waiting time', 5:'Food',
             6:'Food', 7:'Hospitality'}

top_10k = top_10k.replace({'topic':topic_dict})

In [ ]:
top_10k.to_csv("top_10k_mallet.csv")